In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
cwd = os.getcwd()
root_dir = os.path.abspath(os.path.join(cwd, os.pardir))
root_dir

In [ ]:
def read_map(name):
    full_name = os.path.join(root_dir, name)
    if not os.path.isfile(full_name):
        print("El archivo %s no existe"%os.path.basename(full_name))
        return
    try:
        df = pd.read_csv(full_name, delimiter='\\s+', header=None,
                        names=["x", "y", "pot", "ax", "ay"])
        df["r"] = np.sqrt(df["x"]**2 + df["y"]**2)
        df["acc"] = np.sqrt(df["ax"]**2 + df["ay"]**2)
    except Exception as e:
        print("Error al leer el archivo %s"%os.path.basename(full_name))
        print(e)
        df = None
    return df

In [ ]:
mapa = read_map("map.out")
mapa

In [ ]:
if mapa is not None:
    # Create pivot table
    pivot = mapa.pivot_table(values=["pot", "acc", "ax", "ay"], index="y", columns="x")
    copyvot = pivot["pot"].where(pivot["ax"] != 0., 0.)
    pivot

In [ ]:
if mapa is not None:
    # Plot the potential, with pcolormesh
    fig, ax = plt.subplots(figsize=(6,6), dpi=150)

    pcm = ax.pcolormesh(np.log10(copyvot.abs()), vmin=7.0, vmax=7.75)

    # ax.set_title("Potential")
    ax.set_aspect("equal")
    ax.set_xticks([])
    ax.set_yticks([])

    cbar = plt.colorbar(pcm, ax=ax, fraction=0.046, pad=0.04)
    cbar.set_label("$\log_{10}$(pot)")

    # Draw some contours
    ax.contour(np.log10(pivot["pot"].abs()),
               levels=np.linspace(7.1, 7.36, 5),
               colors="k")
    
    fig.tight_layout()

    plt.savefig("potential.png", bbox_inches="tight")

    plt.show()

In [ ]:
if mapa is not None:
    # Plot the potential, with pcolormesh
    fig, axs = plt.subplots(1,3,dpi=150, sharey=True, figsize=(16,4))
    axs[0].pcolormesh(np.log10(pivot["acc"].abs()), vmin=1, vmax=7)
    axs[1].pcolormesh(np.log10(pivot["ax"].abs()), vmin=1, vmax=7)
    p = axs[2].pcolormesh(np.log10(pivot["ay"].abs()), vmin=1, vmax=7)
    fig.colorbar(p, ax=axs.ravel().tolist())
    plt.show()